<a href="https://colab.research.google.com/github/babypotatotang/Introduction-to-DeepLearning/blob/main/8.%20NLP%20Preprocess/02.Torchtext%20Tutorial_Eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 파이토치에서는 텍스트에 대한 여러 추상화 기능을 제공하는 자연어 처리 라이브러리 `torchtext`를 제공함. 
* 제공하는 기능은 다음과 같음. 
    * file load) 다양한 포맷의 코퍼스 로드
    * tokenization) 문장을 단어 단위로 분리
    * vocab) 전체 코퍼스의 단어들을 각각의 고유한 정수로 맵핑 
    * word vector) 단어 집합의 단어들에 고유한 임베딩 벡터를 만들어 줌. 
    * batching) 훈련 샘플들의 배치를 만들어 주며 패딩 작업도 이루어 줌. 
* 전처리 이전에 데이터를 train, val, test로 분리하는 작업을 수행해야 하며, 이후 각 샘플의 단어를 임베딩 벡터로 맵핑해주는 작업(Lookup table)을 수행해야함. 

# **0.실습 전 참고**

In [ ]:
# 0.10.0 버전으로 install 
!pip install torchtext==0.10.0

In [ ]:
# 다음의 방식으로 torchtext를 임포트할 수 있음. 
from torchtext.legacy.data import TabularDataset

# **1. 훈련 데이터와 테스트 데이터로 분리하기**
--- 
- 이번 실습에서 IMDB 리뷰 데이터를 다운 받아 훈련데이터와 테스트 데이터로 분리함 

In [ ]:
import urllib.request
import pandas as pd

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

In [ ]:
df = pd.read_csv('IMDb_Reviews.csv', encoding = 'latin1')
df.head() # 

In [ ]:
print(f'전체 샘플의 개수 : {len(df)}')

In [ ]:
# 상위 25,000개의 행은 train_df에 하위 25,000개의 행은 test_df에 저장 
train_df = df[:25000]
test_df = df[25000:]

In [ ]:
train_df.to_csv("train_data.csv", index = False)
test_df.to_csv("test_data.csv", index = False)

# **2. 필드 정의하기(torchtext.data)**
--- 
* torchtext.data에는 field라는 도구가 제공됨 
* 이 필드를 통해 앞으로 어떤 전처리를 할 것인지 정의함. 

In [ ]:
from torchtext.legacy import data # torchtext.data 임포트

In [ ]:
# 필드 정의
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  batch_first=True,
                  fix_length=20)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

* sequential : 시퀀스 데이터 여부 (True가 기본값) 
* use_vocab : 단어 집합을 만들 것인지 여부 (True가 기본값) 
* tokenize : 어떤 토큰화 함수를 사용할 것인지 지정 (str.split이 기본값) 
* lower : 영어 데이터를 전부 소문자화함 (False가 기본값) 
* batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부 (False가 기본) 
* is_target : 레이블 데이터 여부 (False가 기본값) 
* fix_length : 최대 허용 길이, 그 길이를 맞춰서 패딩(padding) 작업이 진행됨

# **3. 데이터셋 만들기**
--- 

In [ ]:
# 다음의 방식으로 torchtext를 임포트할 수 있음. 
from torchtext.legacy.data import TabularDataset

In [ ]:
train_data, test_data = TabularDataset.splits(
    path = '.', train='train_data.csv', test = 'test_data.csv', format = 'csv', 
    fields = [('text',TEXT), ('label',LABEL)], skip_header = True
)

In [ ]:
print(f'훈련 샘플의 개수 : {len(train_data)}')
print(f'테스트 샘플의 개수 : {len(test_data)}')

In [ ]:
print(vars(train_data[0]))
print(train_data.fields.items())

* TabularDatasets의 fields 인자로 TEXT 필드는 text로 호칭하고, LABEL 필드는 label로 지정하였음. 위 코드를 보았을때, 다음과 같이 구성된 것을 확인할 수 있음. 

# **4. 단어 집합(Vocabulary) 만들기**
---
* 토큰화 전처리를 끝냈다면, 각 단어에 고유한 정수를 맵핑해주는 **정수 인코딩 작업이 필요함 (Integer encoding)** 그리고 전처리를 위해 단어 집합을 만들어 주어야함. 

In [ ]:
# .build_vocab() 도구를 사용하여 단어 집합을 생성 
## min_freq: 단어 집합에 추가 시 단어의 최소 등장 빈도 조건
## max_size: 단어 집합의 최대 크기를 지정  
TEXT.build_vocab(train_data,min_freq = 10, max_size = 10000)

In [ ]:
print(f'단어 집합의 크기: {len(TEXT.vocab)}')

In [ ]:
print(TEXT.vocab.stoi)
print(len(TEXT.vocab.stoi)) #  string to int 

실제로 지정한 단어 집합의 크기는 10,000개 이지만, 생성된 집합의 크기는 총 10,002개로 2개의 토큰이 추가로 생성됨. 이는 `<unk>`와 `<pad` 토큰며, 각각 단어 집합에 없는 단어를 표현하거나 길이를 맞추는 패딩 작업에 사용됨. 

# **5. 토치텍스트의 데이터로더 만들기**
---
* 데이터로더는 데이터셋에서 미니 배치만큼 데이터를 로드하게 만들어주는 역할을 함. 토치텍스트에서는 `Iterator`를 사용하여 만들어줌. 

In [ ]:
from torchtext.legacy.data import Iterator

In [ ]:
batch_size = 5

In [ ]:
train_loader = Iterator(dataset = train_data, batch_size = batch_size)
test_loader = Iterator(dataset=test_data, batch_size = batch_size)

print(f'훈련 데이터의 미니 배치 수 : {len(train_loader)}') # 하나의 배치 당 5개 샘플이 존재함
print(f'테스트 데이터의 미니 배치 수 : {len(test_loader)}')

In [ ]:
batch = next(iter(train_loader)) # 첫번째 미니 배치

In [ ]:
print(type(batch))
print(batch.text)

# **6. <pad> 토큰이 사용되는 경우**
--- 
* 맨 처음 필드를 정의할때, fix_length를 20이 아니라 150으로 정의한다면, 샘플의 뒷부분에 vocab 상 `<pad>` 토큰의 번호였던 숫자인 1로 채워짐. 
* 서로 다른 길이의 샘플들을 동일한 길이로 맞춰주는 작업을 패딩 작업(padding)이라고 함. 